## Exploratory Data Analysis


Before creating a classification algorithm, it is essential to understand the dataset better.

In [7]:
!ls

Achaemenid architecture          Edwardian architecture
American Foursquare architecture Georgian architecture
American craftsman style         Gothic architecture
Ancient Egyptian architecture    Greek Revival architecture
Art Deco architecture            International style
Art Nouveau architecture         Novelty architecture
Baroque architecture             Palladian architecture
Bauhaus architecture             Postmodern architecture
Beaux-Arts architecture          Queen Anne architecture
Byzantine architecture           Romanesque architecture
Chicago school architecture      Russian Revival architecture
Colonial architecture            Tudor Revival architecture
Deconstructivism
